## Import libraries

In [3]:
import sys
sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
import os

os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')

from sem_covid.services.store_registry import StoreRegistry
from pycaret.classification import *
import mlflow
from sem_covid import config

## Load dataset

In [2]:
feature_store = StoreRegistry.es_feature_store()
dataset_x = feature_store.get_features('fs_pwdb_x')
dataset_y = feature_store.get_features('fs_pwdb_y')

100% (1288 of 1288) |####################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (1288 of 1288) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


## Split dataset in train_set and test_set

In [3]:
def py_caret_train_models(dataset_x: pd.DataFrame, dataset_y: pd.DataFrame):
    mlflow.set_tracking_uri(config.MLFLOW_TRACKING_URI)
    for class_name in dataset_y.columns:
        dataset = dataset_x
        dataset[class_name] = dataset_y[class_name].values
        train_data = dataset#.sample(frac=0.70, random_state=786)
        #test_data = dataset.drop(train_data.index)
        train_data.reset_index(inplace=True, drop=True)
        #test_data.reset_index(inplace=True, drop=True)
        experiment = setup(data=train_data,
                           target=class_name,
                           log_experiment=True,
                           experiment_name=f"PyCaret_pwdb_{class_name}",
                           silent=True)
        best_model = compare_models()
        tuned_model = tune_model(best_model,n_iter= 200, choose_better=True,optimize='F1')
        final_model = finalize_model(tuned_model)
        del dataset
        del train_data
        #test_predictions = predict_model(final_model, data=test_data)

In [4]:
py_caret_train_models(dataset_x,dataset_y)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.3956,0.0000,0.1525,0.2421,0.2887,0.2696,0.2885
1,0.3889,0.0000,0.1055,0.2096,0.2659,0.2327,0.2667
2,0.4000,0.0000,0.0823,0.2267,0.2748,0.2464,0.2919
3,0.4000,0.0000,0.1481,0.2674,0.2870,0.2551,0.2940
4,0.4000,0.0000,0.1065,0.2160,0.2670,0.2376,0.2969
5,0.4222,0.0000,0.1347,0.2607,0.2993,0.2845,0.3206
6,0.4000,0.0000,0.1134,0.2905,0.3115,0.2697,0.2919
7,0.3889,0.0000,0.1140,0.2563,0.2814,0.2477,0.2742
8,0.4222,0.0000,0.1508,0.2333,0.2914,0.2824,0.3198
9,0.4111,0.0000,0.1504,0.2501,0.2842,0.2587,0.3095
